# Homework 3

In [1]:
import pandas as pd
import numpy as np
import sympy as sym
from sympy import Symbol, cos, sin, lambdify
import matplotlib.pyplot as plt
import pdb
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

In [2]:
x1  = sym.Symbol('x1')
x2 = sym.Symbol('x2')
function = (5*x1-x2)**4+((x1-2)**2)+(x1-2*x2)+12
f = lambdify([[x1,x2]], function, 'numpy')
# plot the function
f_2 = lambdify([[x1,x2]], function, 'numpy')

In [4]:
x = np.arange(6.0,7.0,0.1)
y = np.arange(32.0,33.0,0.1)
X,Y = meshgrid(x, y) # grid of point
Z = f_2(X,Y) # evaluation of the function on the grid

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, 
                      cmap=cm.RdBu,linewidth=0, antialiased=False)

ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

## Cyclic Coordinate Search

In [3]:
def BisectionMethod(f, a=-100,b=100,epsilon=0.005) :
    iteration=0
    while (b - a) >= epsilon:
        x_1 = (a + b) / 2
        fx_1 = f(x_1)
        if f(x_1 + epsilon) <= fx_1:
            a = x_1
        else:
            b = x_1
        iteration+=1
    x_star = (a+b)/2
    return x_star

In [47]:
def ExactLineSearch(f, x0, d):
    alpha = sym.Symbol('alpha')
    function_alpha = f(np.array(x0)+alpha*np.array(d))
    f_alp = lambdify(alpha, function_alpha, 'numpy')
    alp_star = BisectionMethod(f_alp)
    return alp_star

In [44]:
def CyclicCoordinateSearch(f, x0, epsilon):
    """
    Cyclic Coordinate Search method.
    Parameters
    ----------
    f : lambda expression The function to be evaluated
    x0 : numpy.array Starting coordinates
    epsilon : float Epsilon value determined to check optimality
    Returns
    ---------
    x1 : numpy.array The found x* vector
    k : integer Number of iterations
    """
    x0 = np.array(x0)
    x_array = [x0]
    k = 0
    n = len(x0)
    res_array = []
    while(True):
        for j in range(n):
            x_now = x_array[k]
            d = np.zeros(n)
            d[j] = 1
            # exact line search:
            # minimize f(x0+alpha*d)
            alpha = ExactLineSearch(f, x_now, d)
            x_next = x_now + alpha*d
            x_array.append(x_next)
            res_array.append([k, np.array2string(x_now,precision=4), f(x_now), str(d), alpha, np.array2string(x_next,precision=4)])
            k += 1
        if(np.linalg.norm(x_array[k]-x_array[k-1]) < epsilon):
            result_table = pd.DataFrame(res_array, columns=['k' ,'x^k', 'fx^k', 'd^k', 'a^k', 'x^k+1'])
            return x_array[k], k, result_table

In [46]:
x, k, output = CyclicCoordinateSearch(f,[-8,10],0.01)

## Hook & Jeeves Method

In [ ]:
# code goes to here

## Simplex Search

In [6]:
def SimplexSearch():
    x=np.zeros(shape=(3,2))
    x_1=np.array([-2,15])#initial
    x_2=np.array([-8,10])#initial
    x_3=np.array([0,0])#initial
    x[0]=x_1
    x[1]=x_2
    x[2]=x_3 
    def compute_f_values(a):
        f_values=np.zeros(a.shape[0])    
        for i in range(a.shape[0]):
            f_values[i]=f(a[i])
        return f_values    
    epsilon=1
    alpha=1
    beta=0.5
    gamma=2
    
    while(True):    
        sum_value=0
        f_values=compute_f_values(x)#function values of x_matrix
        x_h=x[np.argmax(f_values)] #the worst point
        x_l=x[np.argmin(f_values)] #the best point
        index_to_go=np.argmax(f_values) #index of x_h
        mean_x=np.delete(x, index_to_go, 0)#x matrix rather than x_h
        x_mean=np.mean(mean_x,axis=0)#compute the mean of x matrix rather than x_h
        f_values_mean_x=compute_f_values(mean_x)#function values of x_matrix except x_h
        
        x_r=x_mean+alpha*(x_mean-x_h) #reflection
        
        if f(x_l)>f(x_r): #the reflected point x_r happens to be better than the current best  
            x_e=x_mean+gamma*(x_r-x_mean) #Expansion
            if f(x_r)>f(x_e): #the expanded point x_e happens to be better than the current best x_r
                x[np.argmax(f_values)]=x_e
            else:             #the expanded point is not better than x_r so we replace x_h with x_r
                x[np.argmax(f_values)]=x_r
        
        else:
            if (np.max(f_values_mean_x))>=f(x_r):
                x[np.argmax(f_values)]=x_r
            else:
                if f(x_h)>f(x_r):
                    x_h_prime=x_r
                else:
                    x_h_prime=x_h
                
                x_c=x_mean+beta*(x_h_prime-x_mean) #contraction
                if f(x_c) <= f(x_h):
                    x[np.argmax(f_values)]=x_c
                else:
                    for i in range(3):
                        x[i]=x[i]+0.5*(x_l-x[i]) #shrink operation
        
        for i in range(3):
            sum_value+=(f(x[i])-f(x_mean))**2
        print(sum_value)
        if np.sqrt(sum_value)<epsilon:
            break
                
    return x_mean

In [7]:
SimplexSearch()